<a href="https://colab.research.google.com/github/fzanart/GHDomains/blob/main/Refactoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk import download
ss = SnowballStemmer('english')
from spacy import load
from spacy.matcher import Matcher
from tqdm import tqdm
nlp = load("en_core_web_sm", exclude=["tok2vec", "tagger", "parser",'senter', "attribute_ruler", "lemmatizer", 'ner'])
# from google.colab import drive
# drive.mount('/content/drive/')
# chi-squared test with similar proportions
from scipy.stats import chi2_contingency
from scipy.stats import chi2
import numpy as np
download('punkt')

[nltk_data] Downloading package punkt to /Users/francisco/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import multiprocessing as mp
pool = mp.Pool(mp.cpu_count())
mp.cpu_count()

8

In [3]:
directory = '/content/drive/MyDrive/GHDomains/Downloaded_repos/'
local_directory = '/Volumes/GoogleDrive/My Drive/GHDomains/Downloaded_repos/'
directory = local_directory

In [4]:
word_list = ['refactor', 'restruct', 'clean', 'not used', 'unused', 'reformat', 'import', 'remove', 'replace', 'split', 'reorg', 'rename', 'move']

matcher = Matcher(nlp.vocab)

# Add patterns
patterns= [[{"LOWER": "refactor"}],
            [{"LOWER": "restruct"}],
            [{"LOWER": "clean"}],
            [{"LOWER": "not"},{"LOWER": "use"}],    # used      steam = use
            [{"LOWER": "unus"}],                    # unused    steam = unus
            [{"LOWER": "reformat"}],                ## steam = reform
            [{"LOWER": "import"}],
            [{"LOWER": "remov"}],                   # remove    steam = remov
            [{"LOWER": "replac"}],                  # replace   steam = replac
            [{"LOWER": "split"}],                   ## split     steam = splite / split
            [{"LOWER": "reorg"}],                   # reorg     steam = reorg
            [{"LOWER": "renam"}],                   # rename    steam = renam
            [{"LOWER": "move"}]]

matcher.add("refactoring_pattern", patterns)

In [5]:
def word_matcher(text):
    
    try:
        with nlp.disable_pipes():
        # Tokenize the commit sentence:
            text = word_tokenize(text)
        # apply steam technique
            text = ' '.join([ss.stem(word) for word in text])
        # select matching words
            text = nlp(text)
            words = matcher(text)
    except TypeError:
        return 0

    if len(words)>0:

        return 1

    else:

        return 0

In [6]:
repos_ids = [file_name for file_name in os.listdir(directory) if file_name.endswith('.json') and not file_name.startswith('all_repos')]
repos_ids = list(set(repos_ids))
len(repos_ids)

889

In [7]:
matches = []
to_rescan = []

for filename in tqdm(repos_ids):
      try:
          aux_reader = pd.read_json(directory+filename)
          commits    = aux_reader['Messages:'].size
          match      = aux_reader['Messages:'].apply(word_matcher).sum()
          filename   = filename.replace(':', '/').replace('.json', '')
          matches.extend([(filename, commits, match)])
      except:
          print('None commits found in: ', filename)
          filename   = filename.replace(':', '/')
          filename   = filename.replace('.json', '')
          to_rescan.extend([filename])


df = pd.DataFrame(matches, columns =['Name', 'Commits', 'Matches'])
df

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 889/889 [18:29<00:00,  1.25s/it]


,Name,Commits,Matches
0,dragen1860/Deep-Learning-with-TensorFlow-book,151,2
1,Tencent/MMKV,1039,82
2,webtorrent/webtorrent,3090,207
3,apache/skywalking,6952,690
4,qyuhen/book,66,0
...,...,...,...
884,fengdu78/lihang-code,45,0
885,aui/art-template,598,26
886,go-redis/redis,2019,192
887,strapi/strapi,20568,2178


In [8]:
df_path = '/content/drive/MyDrive/GHDomains/Resources/new_popular_df_predicted.csv'
df_path = '/Volumes/GoogleDrive/My Drive/GHDomains/Resources/new_popular_df_predicted.csv'

new_popular_df = pd.read_csv(df_path, usecols=['Name','Predicted_labels'])
new_popular_df

,Name,Predicted_labels
0,vuejs/devtools,Web libraries and frameworks
1,futurice/android-best-practices,Documentation
2,microsoft/Web-Dev-For-Beginners,Documentation
3,airbnb/react-sketchapp,Web libraries and frameworks
4,eugeneyan/applied-ml,Documentation
...,...,...
436,railsware/upterm,Software tools
437,matryer/xbar,Software tools
438,thedaviddias/Front-End-Checklist,Documentation
439,electronicarts/CnC_Remastered_Collection,Documentation


In [9]:
join_data = pd.merge(new_popular_df, df, on ='Name', how ='inner')
join_data['Non refactoring commits'] = join_data['Commits'] - join_data['Matches']

columns = ['Name','Domain','Total commits', 'Refactoring commits', 'Non refactoring commits']
join_data.columns = columns
join_data = join_data[['Name','Domain', 'Refactoring commits', 'Non refactoring commits','Total commits']]
join_data

,Name,Domain,Refactoring commits,Non refactoring commits,Total commits
0,vuejs/devtools,Web libraries and frameworks,246,1723,1969
1,futurice/android-best-practices,Documentation,31,216,247
2,microsoft/Web-Dev-For-Beginners,Documentation,60,1205,1265
3,airbnb/react-sketchapp,Web libraries and frameworks,123,657,780
4,eugeneyan/applied-ml,Documentation,22,377,399
...,...,...,...,...,...
436,railsware/upterm,Software tools,446,2552,2998
437,matryer/xbar,Software tools,96,935,1031
438,thedaviddias/Front-End-Checklist,Documentation,82,612,694
439,electronicarts/CnC_Remastered_Collection,Documentation,1,11,12


In [10]:
df = join_data.groupby(['Domain']).sum()
df

,Refactoring commits,Non refactoring commits,Total commits
Domain,,,
Application & System software,26944,221592,248536
Documentation,14659,171339,185998
Non-web libraries and frameworks,39691,218165,257856
Software tools,95763,684374,780137
Web libraries and frameworks,40523,280503,321026


In [11]:
domains = join_data['Domain'].unique()

for domain in domains:
    print(domain, ':')
    data1 = df.loc[domain]
    data2 = df.loc[domains[domains != domain]].sum()
    aux = np.column_stack((data1, data2))[:2].T
    print(aux)
    stat, p, dof, expected = chi2_contingency(aux)
    print('dof=%d' % dof)
    print(expected)
    # interpret test-statistic
    prob = 0.95
    critical = chi2.ppf(prob, dof)
    print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))
    if abs(stat) >= critical:
        print('Dependent (reject H0)')
    else:
        print('Independent (fail to reject H0)')
    # interpret p-value
    alpha = 1.0 - prob
    print('significance=%.3f, p=%.3f' % (alpha, p))
    if p <= alpha:
        print('Dependent (reject H0)')
    else:
        print('Independent (fail to reject H0)')
    print('=========================================')

Web libraries and frameworks :
[[  40523  280503]
 [ 177057 1295470]]
dof=1
[[  38944.3953315  282081.6046685]
 [ 178635.6046685 1293891.3953315]]
probability=0.950, critical=3.841, stat=88.643
Dependent (reject H0)
significance=0.050, p=0.000
Dependent (reject H0)
Documentation :
[[  14659  171339]
 [ 202921 1404634]]
dof=1
[[  22563.84106854  163434.15893146]
 [ 195016.15893146 1412538.84106854]]
probability=0.950, critical=3.841, stat=3515.864
Dependent (reject H0)
significance=0.050, p=0.000
Dependent (reject H0)
Software tools :
[[ 95763 684374]
 [121817 891599]]
dof=1
[[ 94640.19655957 685496.80344043]
 [122939.80344043 890476.19655957]]
probability=0.950, critical=3.841, stat=26.806
Dependent (reject H0)
significance=0.050, p=0.000
Dependent (reject H0)
Application & System software :
[[  26944  221592]
 [ 190636 1354381]]
dof=1
[[  30150.46830509  218385.53169491]
 [ 187429.53169491 1357587.46830509]]
probability=0.950, critical=3.841, stat=450.371
Dependent (reject H0)
signifi